# Contexts

## Computational contexts

### Functor

#### `<$>`

In [124]:
from Contexts.Functor import ListFunctor

a=ListFunctor([1,2,3,4])
a.fmap(float).fmap(str)

['1.0', '2.0', '3.0', '4.0']

In [125]:
from Contexts.Functor import TupleFunctor

a=TupleFunctor(tuple([1,2,3,4]))
a.fmap(float).fmap(str)

('1.0', '2.0', '3.0', '4.0')

In [126]:
print(a)

(1, 2, 3, 4)


In [127]:
a=ListFunctor([])
a.fmap(float)

[]

In [128]:
a=TupleFunctor(())
a.fmap(float)

()

#### `<$`: map replace-by

In [129]:
def const(a,b):
    return a

In [130]:
from functools import partial
def replaceby(a):
    return partial(const,a)

list(map(replaceby([1,2]),[1,2,3,4]))

[[1, 2], [1, 2], [1, 2], [1, 2]]

In [131]:
a=ListFunctor([1,2,3,4])
a.fmap(replaceby([1,2,3]))

[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]

### Applicative

In [132]:
from Contexts.Applicative import ListApplicative

a=ListApplicative([1,2,3,4])
a.ap([lambda x: -x]).ap([float,abs])

[-1.0, -2.0, -3.0, -4.0, 1, 2, 3, 4]

In [133]:
a.then([1,2])

[1, 2, 1, 2, 1, 2, 1, 2]

In [134]:
a.pure(3).ap([float,abs])

[3.0, 3]

In [135]:
from Contexts.Applicative import pure, liftA2, ap

pure([3])

[3]

In [136]:
from operator import add

list(liftA2(add,[1,2],[3,4]))

[4, 5, 5, 6]

In [137]:
from Basic.Monoid import *

a.ap(pure([partial(add,1)]))

[2, 3, 4, 5]

In [138]:
a.ap([partial(add,1)])

[2, 3, 4, 5]

In [139]:
a.ap([partial(y,x) for x,y in zip([1,2],[add]*2)])

[2, 3, 4, 5, 3, 4, 5, 6]

In [140]:
a=ListApplicative([a])

In [141]:
a.ap([lambda x:x[:2],lambda x:x[2:]])

[[1, 2], [3, 4]]

In [142]:
type(a+a)

list

In [143]:
b = mappend(a,a)
type(b)

Contexts.Applicative.ListApplicative

#### Empty applicative

In [144]:
c=mempty(b)

In [145]:
type(c)

Contexts.Applicative.ListApplicative

The `pure` method still work for an empty applicative:

In [146]:
c.pure(3)

[3]

In [147]:
c.then([1,2])

[]

Beautiful! So in the empty applicative context, replacing other contents still resulting in empty.

### Monad

In [148]:
from Contexts.Monad import ListMonad

a=ListMonad([1,2,3,4])
a.ap([lambda x: -x]).ap([float,abs])

[-1.0, -2.0, -3.0, -4.0, 1, 2, 3, 4]

In [149]:
a.fmap(lambda x: [x-1,x,x+1])

[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5]]

In [150]:
a.bind(lambda x: [x-1,x,x+1])

[0, 1, 2, 1, 2, 3, 2, 3, 4, 3, 4, 5]

In [151]:
a.bind(lambda x: [x-1,x,x+1]).bind(lambda x: [x] if (x%2==0) else [])

[0, 2, 2, 2, 4, 4]

In [152]:
type(mappend(a,a))

Contexts.Monad.ListMonad

## Interator contexts

### Foldable

In [153]:
from Contexts.Foldable import ListFoldable, foldr, foldr_r, scanl, scanl_r, scanr, scanr_r

a=ListFoldable([1,2,3,4])
a.foldl((lambda acc,x: acc+x),0)

10

In [154]:
a.foldr((lambda x,acc: x+acc),0)

10

In [155]:
foldr((lambda x,acc:x+acc),(1,2,3,4),0)

10

In [156]:
next(foldr_r((lambda x,acc:x+acc),(1,2,3,4),0))

10

In [157]:
list(scanl((lambda acc,x: acc+x),0,(1,2,3,4)))

[1, 3, 6, 10]

In [158]:
def leftEdge(n,acc,x): return (acc[1][-n:],x)
def rightEdge(n,x,acc): return x+(acc[1][:n],)

In [159]:
a=[[1,2,3,4],[5,6,7,8]]
r2=partial(rightEdge,2)
l2=partial(leftEdge,2)
foo=scanl(l2,[[],[-1,0]],a)
print(foo)

[([-1, 0], [1, 2, 3, 4]), ([3, 4], [5, 6, 7, 8])]


In [160]:
scanr(r2,foo,[[],[9,10],[]])[::-1]

[([-1, 0], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [9, 10])]

In [161]:
print(a)

[[1, 2, 3, 4], [5, 6, 7, 8]]


In [162]:

foo=list(scanl_r(l2,[[],[-1,0]],a))
print(foo)

[([-1, 0], [1, 2, 3, 4]), ([3, 4], [5, 6, 7, 8])]


In [163]:
print(a)

[[1, 2, 3, 4], [5, 6, 7, 8]]


In [164]:
list(scanr_r(r2,foo,[[],[9,10],[]]))[::-1]

[([-1, 0], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [9, 10])]